<a href="https://colab.research.google.com/github/HimanshuPathak2725/googleColab/blob/main/Copy_of_langchain_chroma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain chromadb tiktoken pypdf langchain-huggingface langchain-community langchain-chroma

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
from langchain.schema import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [4]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [5]:
vector_store = Chroma(
    embedding_function=HuggingFaceEmbeddings(
        model_name="BAAI/bge-base-en",
    model_kwargs={"device": "cuda"}
    ),
    persist_directory='my_chroma_db',
    collection_name='sample'
)

In [6]:
# add documents
vector_store.add_documents(docs)

['efc025ae-768e-4c77-a7f7-219a2867a503',
 'cffd7902-51ca-40c1-9734-55710bd86a5e',
 'cabee804-9d51-4ceb-8211-2fc43a6fcc82',
 '4db11a96-a1a0-4891-be3d-73e51502c1e5',
 '34c5968c-8890-4117-8291-93b99e2f3a20']

In [7]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['efc025ae-768e-4c77-a7f7-219a2867a503',
  'cffd7902-51ca-40c1-9734-55710bd86a5e',
  'cabee804-9d51-4ceb-8211-2fc43a6fcc82',
  '4db11a96-a1a0-4891-be3d-73e51502c1e5',
  '34c5968c-8890-4117-8291-93b99e2f3a20'],
 'embeddings': array([[-0.02124029, -0.03699498, -0.00824418, ...,  0.00811559,
         -0.01111715, -0.01759077],
        [-0.06620909, -0.05114624, -0.00162179, ...,  0.03091763,
          0.01865165, -0.04055971],
        [-0.03133981, -0.02786523,  0.00987367, ...,  0.00155221,
          0.00899096,  0.00709367],
        [-0.04911939, -0.05274933, -0.01010442, ...,  0.04359226,
          0.03046085, -0.0145939 ],
        [-0.00258785, -0.01945574, -0.0340637 , ...,  0.02387643,
          0.00604695, -0.01120152]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [8]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(id='4db11a96-a1a0-4891-be3d-73e51502c1e5', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(id='efc025ae-768e-4c77-a7f7-219a2867a503', metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.')]

In [9]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(id='4db11a96-a1a0-4891-be3d-73e51502c1e5', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.39887550473213196),
 (Document(id='efc025ae-768e-4c77-a7f7-219a2867a503', metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.'),
  0.45155709981918335)]

In [10]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(id='cabee804-9d51-4ceb-8211-2fc43a6fcc82', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.49531152844429016),
 (Document(id='34c5968c-8890-4117-8291-93b99e2f3a20', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.5379127264022827)]

In [11]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)


In [12]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['efc025ae-768e-4c77-a7f7-219a2867a503',
  'cffd7902-51ca-40c1-9734-55710bd86a5e',
  'cabee804-9d51-4ceb-8211-2fc43a6fcc82',
  '4db11a96-a1a0-4891-be3d-73e51502c1e5',
  '34c5968c-8890-4117-8291-93b99e2f3a20'],
 'embeddings': array([[-0.02124029, -0.03699498, -0.00824418, ...,  0.00811559,
         -0.01111715, -0.01759077],
        [-0.06620909, -0.05114624, -0.00162179, ...,  0.03091763,
          0.01865165, -0.04055971],
        [-0.03133981, -0.02786523,  0.00987367, ...,  0.00155221,
          0.00899096,  0.00709367],
        [-0.04911939, -0.05274933, -0.01010442, ...,  0.04359226,
          0.03046085, -0.0145939 ],
        [-0.00258785, -0.01945574, -0.0340637 , ...,  0.02387643,
          0.00604695, -0.01120152]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [13]:
# delete document
vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

In [14]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['efc025ae-768e-4c77-a7f7-219a2867a503',
  'cffd7902-51ca-40c1-9734-55710bd86a5e',
  'cabee804-9d51-4ceb-8211-2fc43a6fcc82',
  '4db11a96-a1a0-4891-be3d-73e51502c1e5',
  '34c5968c-8890-4117-8291-93b99e2f3a20'],
 'embeddings': array([[-0.02124029, -0.03699498, -0.00824418, ...,  0.00811559,
         -0.01111715, -0.01759077],
        [-0.06620909, -0.05114624, -0.00162179, ...,  0.03091763,
          0.01865165, -0.04055971],
        [-0.03133981, -0.02786523,  0.00987367, ...,  0.00155221,
          0.00899096,  0.00709367],
        [-0.04911939, -0.05274933, -0.01010442, ...,  0.04359226,
          0.03046085, -0.0145939 ],
        [-0.00258785, -0.01945574, -0.0340637 , ...,  0.02387643,
          0.00604695, -0.01120152]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca